# Import Libaries

In [1]:
try:
    from elasticsearch import Elasticsearch, RequestsHttpConnection
    from elasticsearch.client import IndicesClient
    from elasticsearch.helpers import reindex
except:
    !pip install 'elasticsearch<7.14.0'
    from elasticsearch import Elasticsearch, RequestsHttpConnection
    from elasticsearch.client import IndicesClient
    from elasticsearch.helpers import reindex
    
try:
    from requests_aws4auth import AWS4Auth
except:
    !pip install requests_aws4auth
    from requests_aws4auth import AWS4Auth
    
import boto3
import botocore
import os
import pandas as pd
import numpy as np
import re
import time
# import datetime
from datetime import datetime
import dateutil.parser as parser

## Send HTTPS request to ES

In [2]:
service = 'es'
ss = boto3.Session()
credentials = ss.get_credentials()
region = ss.region_name

#ES domain
host = "vpc-dusstac-dussta-1n8niblaemqb-otcfzpck6s7czlgni6gxcb4rru.us-east-1.es.amazonaws.com"

awsauth = AWS4Auth(credentials.access_key, credentials.secret_key,
                   region, service, session_token=credentials.token)

# set up ES client for future API calls
es = Elasticsearch(
    hosts=[{'host': host, 'port': 443}],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

### Test Connection to AWS ES Service
Testing the connection to ES using a HTTP request body and parameters. `searchBody` below is the same as the search body in dusstackapiprocessor. The index in DUS is called `textract`.

In [3]:
es

<Elasticsearch([{'host': 'vpc-dusstac-dussta-1n8niblaemqb-otcfzpck6s7czlgni6gxcb4rru.us-east-1.es.amazonaws.com', 'port': 443}])>

In [4]:
!curl -H 'Content-Type: application/json' -X GET https://vpc-dusstac-dussta-1n8niblaemqb-otcfzpck6s7czlgni6gxcb4rru.us-east-1.es.amazonaws.com/textract?pretty

{
  "textract" : {
    "aliases" : { },
    "mappings" : {
      "properties" : {
        "5" : {
          "properties" : {
            " this authorization shall be in force and effect until 12/31/2020 (date event), at which time this authorization expires" : {
              "type" : "text",
              "fields" : {
                "keyword" : {
                  "type" : "keyword",
                  "ignore_above" : 256
                }
              }
            }
          }
        },
        "anatomy" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "bucket" : {
          "type" : "text",
          "fields" : {
            "keyword" : {
              "type" : "keyword",
              "ignore_above" : 256
            }
          }
        },
        "by" : {
          "type" : "text",
          "fields" : {
            "keyword" : {


In [16]:
# search query
# keyword = "What is diabetes\?"
# keyword = "What are the types of diabetes\?"
# keyword = "What are the types of contracts\?"
keyword = "contracts"

# size of the result fragment returned by ES
ES_HIGHLIGHT_FRAGMENT_SIZE = 200

# HTTP request parameters that will be sent to ES API
searchBody = {
    # set up for string based queries
    "query" : {
        "query_string": {
            "query": keyword
        }
    },
    # set up for result fragments that are returned by ES
    "highlight" : {
        "fields" : {
            "content" : { "pre_tags" : [""], "post_tags" : [""] },
        },
        "fragment_size" : ES_HIGHLIGHT_FRAGMENT_SIZE,
        "require_field_match": False
    }
}

# make a query against existing index in ES via call to API
output = es.search(
    index='textract',
    size=1000,
    body=searchBody,
    _source = True,
    filter_path=[
        'hits.hits._id',
        'hits.hits._source',
        'hits.hits.highlight',
        'hits.hits._score',
        'hits.hits.date'
    ],
    request_timeout=5 # this isn't what limits to 5 fragments
)

# Download and save
df_res = pd.DataFrame(columns=['doc_path','doc_name','score', 'highlight'])

# parse results from search; returns 5 fragments from each source
n_results = len(output["hits"]["hits"])
print(f"The Elasticsearch query returned {n_results} results.\n")

print(f"Example output:\n\n")
for i,x in enumerate(output["hits"]["hits"]):
    newrow = {'doc_path':os.path.split(x['_source']['name'])[0],
              'doc_name':os.path.split(x['_source']['name'])[1],
              'score':x['_score'],
              'highlight':''}
    for j,highlight in enumerate(x['highlight']['content']):
        print(f"SOURCE {i}: HIGHLIGHT {j}")
        highlight = highlight.replace("\n"," ")
        print(f"{highlight}\n")
        newrow['highlight'] = highlight
        df_res.loc[df_res.shape[0],:] = newrow
        
# save the file
# flnm = 'query_' + re.sub('\W+','', keyword.replace(" ","_").lower()) + '_1.csv'
# df_res.to_csv(flnm, index=False)

The Elasticsearch query returned 18 results.

Example output:


SOURCE 0: HIGHLIGHT 0
Different types diabetic nephropathy, is kidney disease of nerve damage affect different parts of caused by diabetes. You can help protect your body.

SOURCE 0: HIGHLIGHT 1
& Dementia & Sleep Apnea & Diabetes Diabetes Diabetes Diabetes Depression NIH Diabetes is linked High blood People who have is common among to some types of glucose increases sleep apnea NIH people with

SOURCE 0: HIGHLIGHT 2
Many the chance of - -when you stop chronic, or long- risk factors for developing breathing for term, illness such cancer and for dementia NIHG. short periods as diabetes. diabetes are the Tell your doctor

SOURCE 0: HIGHLIGHT 3
This content is provided as a service of the National Institute of Diabetes and Digestive and Kidney Diseases (NIDDK), part of the National Institutes of Health.

SOURCE 0: HIGHLIGHT 4
Contact Us The National Institute of Diabetes and Digestive and Kidney Diseases Health Information Ce

In [17]:
len(output['hits']['hits'])

18

## Download Test

In [103]:
# set up a query and download the contents
# use output from previous query
# why does highlight only return 5?

df_res = pd.DataFrame(columns=['doc_path','doc_name','score', 'highlight'])

# parse results from search; returns 5 fragments from each source
n_results = len(output["hits"]["hits"])
print(f"The Elasticsearch query returned {n_results} results.\n")

print(f"Example output:\n\n")
for i,x in enumerate(output["hits"]["hits"]):
    newrow = {'doc_path':os.path.split(x['_source']['name'])[0],
              'doc_name':os.path.split(x['_source']['name'])[1],
              'score':x['_score'],
              'highlight':''}
#     df_res.loc[df_res.shape[0],:] = newrow
    for j,highlight in enumerate(x['highlight']['content']):
        highlight = highlight.replace("\n"," ")
        newrow['highlight'] = highlight
        df_res.loc[df_res.shape[0],:] = newrow

The Elasticsearch query returned 17 results.

Example output:




In [104]:
df_res.to_csv('save_query_types_of_diabetes.csv', index=False)

# Examine the Index from stackoverflow

In [145]:
res = es.search(
    index='originaltextractindex',
    body= {'size':10000,
           'query': {
               'match_all':{}
           }
          },
)

In [146]:
for i,doc in enumerate(res['hits']['hits']):
    print(i, createDate(), doc['_source']['name'])

0 [datetime.datetime(2014, 3, 15, 6, 25, 10), '+0300'] public/85f5715b-c972-41e7-a554-9174d9799fca/32100355.pdf
1 [datetime.datetime(2009, 4, 18, 11, 48, 38), '-0430'] public/a3744816-6066-4f34-b066-5411a4bf4555/Creatine_Kinase_-_MedlinePlus_Medical_Test.pdf
2 [datetime.datetime(2018, 9, 3, 14, 48, 46), '-0030'] public/23d1a9d2-4fd3-40b0-869c-f5cb8c9a1958/KS United Healthcare Contract.pdf
3 [datetime.datetime(2017, 11, 21, 19, 31, 50), '+0500'] public/504aa6d9-ad31-4e7d-9a0c-18fbda4bb45e/KS Aetna Contract.pdf
4 [datetime.datetime(1994, 11, 4, 3, 13, 46), '-0800'] public/65b27304-1f38-4227-af57-ead1b1d27266/Preventing_Diabetes_Problems_NIDDK.pdf
5 [datetime.datetime(2003, 5, 10, 6, 17, 28), '+0100'] public/a55db906-83f2-45e2-99de-5d80293f009d/KS Sunflower Health Plan Contract.pdf
6 [datetime.datetime(2010, 10, 11, 17, 8, 47), '+0800'] public/4c9040b9-cf33-4a81-84cd-ec057c328afe/KS Aetna Contract.pdf


### Finding pdf date modified

In [7]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument

fp = open('local_file', 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)

print(doc.info)  # The "Info" metadata

[{'CreationDate': b"D:20141015162659-06'00'", 'Creator': b'Adobe Acrobat Pro 10.0.0', 'ModDate': b"D:20180417152548-07'00'", 'Producer': b'Adobe Acrobat 10.0 Paper Capture Plug-in', 'Title': b''}]


In [42]:
def convertPdfDatetime(dt, dtformat = None):
    if dtformat is None:
        dtformat = "%Y%m%d%H%M%S"
    print(dt)
    clean = re.split('([+ -])', dt.decode("utf-8").replace("D:",""))
    gmt = datetime.strptime(clean[0], dtformat)
    timezone = clean[1] + clean[2]
    return gmt, timezone
    # time zone parsing?

In [43]:
convertPdfDatetime(doc.info[0]['ModDate'])

b"D:20180417152548-07'00'"


(datetime.datetime(2018, 4, 17, 15, 25, 48), "-07'00'")

1. year
2. month
1. date
1. hour
1. min
1. sec
1. sign
1. hr sign
1. half hr sign

In [32]:
def createDate(dtformat = None, return_type = None):
    # a mess
    if dtformat is None:
        dtformat = "%Y%m%d%H%M%S"
    
    if return_type is None:
        return_type = 'iso' # raw/tuple/iso

    # regenerated only when this cell is run
    randconfig = {
                  'year': np.random.randint(2000,2022),
                  'month': np.random.randint(1, 12),
                  'day': np.random.randint(1,28),
                  'hour': np.random.randint(1, 24),
                  'minute': np.random.randint(1, 60),
                  'second': np.random.uniform(1, 60),
                  'sign': np.random.choice(['-','+']),
                  'tz_hr': np.random.randint(0, 12),
                  'tz_min': np.random.choice([0, 30]),
                 }
    
    if return_type.lower() == 'iso':
        # pattern = '%04d-%02d-%02dT%02d:%02d:%02.3dZ%s%02d%02d' # trying to do ISO directly, superseded by dateutils.parser
        pattern = '%04d%02d%02d %02d:%02d:%02.3d%s%02d%02d'
        dtstr = pattern %(randconfig['year'], randconfig['month'], randconfig['day'], randconfig['hour'], randconfig['minute'], randconfig['second'], randconfig['sign'], randconfig['tz_hr'], randconfig['tz_min'])
        dtstr = parser.parse(dtstr).isoformat()
    else:
        # pattern = '%d%02d%02d%02d%02d%02d%s%02d%02d'
        pattern = '%d%02d%02d%02d%02d%02.3f%s%02d%02d' # seconds upto ms
        dtstr = pattern %(randconfig['year'], randconfig['month'], randconfig['day'], randconfig['hour'], randconfig['minute'], randconfig['second'], randconfig['sign'], randconfig['tz_hr'], randconfig['tz_min'])
    
    if return_type.lower() == 'tuple':
        clean = re.split('([+ -])', dtstr)
        gmt = datetime.strptime(clean[0], dtformat)
        timezone = clean[1] + clean[2]
        return [gmt, timezone]
    else:
        return dtstr
    

In [35]:
createDate(return_type='iso')

'2008-01-18T06:28:48+04:00'

In [ ]:
randconfig

In [24]:
# text = 'Thu, 16 Dec 2010 12:14:05.8 +0000'
text = '20101216 12:14:05.89+0000'
date = parser.parse(text)
print(date.isoformat())
# 2010-12-16T12:14:05+00:00

2010-12-16T12:14:05.890000+00:00


In [171]:
dtstr

'20050621021021-0600'

In [13]:
parser.parse(createDate(return_type='raw')).isoformat()

ParserError: Unknown string format: 20040726094841.734+0600

In [16]:
dtstr2 = createDate(return_type='raw')
datetime.strptime(dtstr2, '%Y-%m-%dT%H%M%SS.DDD%s%02d%02d') # YYYY-MM-DDTHH:MM:SS.DDDZ

ValueError: 's' is a bad directive in format '%Y-%m-%dT%H%M%SS.DDD%s%02d%02d'

## Update textract into a new index

In [141]:
doc=res['hits']['hits'][0]['_source']
doc["moddate"] = createDate()

In [ ]:
doc

In [153]:
ind_client.exists(index="originaltextractindex")

NameError: name 'ind_client' is not defined

In [ ]:
if not ind_client.exists(index="originaltextractindex"):
    
    print("Index 'originaltextractindex' does not exist, creating...")
    ind_client.create(
        index="originaltextractindex",
        body={
            "settings": {
                "index": {
                    "number_of_shards": 2
                }
            },
            "mappings":{
                "properties":{
                "date":{
                    "type": "date",
                    "format": "M'/'dd'/'yyyy||date||year||year_month||dd MMM yyyy||dd'/'MM'/'yyyy||yyyy'/'MM'/'dd||dd'/'MM'/'YY||year_month_day||MM'/'dd'/'yy||dd MMM||MM'/'yyyy||M-dd-yyyy||MM'/'dd'/'yyyy||M||d'/'MM'/'yyyy||MM'/'dd'/'yy"
                    }
                }
            }
        }
    )
        
    reindex(
        client = es, # original ES client specified in the beginning
        source_index = "textract", # original index with documents we want to copy
        target_index = "originaltextractindex" # new index where documents will be copied to
    ) 

In [79]:
res['hits']['hits'][0]['_source'].keys()

dict_keys(['documentId', 'name', 'bucket', 'content', 'organization', 'other', 'location', 'date', 'person', 'title', 'quantity', 'event', 'protected_health_information', 'medical_condition', 'test_treatment_procedure', 'medication', 'subject', 'to', 'from', 'christa capozzola', 'circular'])

In [58]:
res['hits']['hits'][10]['_source']['name']

'public/a55db906-83f2-45e2-99de-5d80293f009d/KS Sunflower Health Plan Contract.pdf'

## Direct S3 to find lastmodified

In [ ]:
s3 = boto3.client('s3')
bucket_name = res['hits']['hits'][10]['_source']['bucket']
print(bucket_name)
objects = s3.list_objects(Bucket=bucket_name)
print(len(objects['Contents']))

dusstack-dusstackdocumentss3bucketbpxgdwfrjv578tc-1vu4fuke61l7n
1000


In [81]:
objects.keys()

dict_keys(['ResponseMetadata', 'IsTruncated', 'Marker', 'Contents', 'Name', 'Prefix', 'MaxKeys', 'EncodingType'])

In [89]:
os.path.splitext(objects['Contents'][10]['Key'])[1][1:]

'csv'

In [ ]:
for doc in objects['Contents']:
    print(doc['Key'], doc['LastModified']) # but this lastmodified is when modified IN the bucket

In [91]:
for doc in objects['Contents'][0:1]:
    extension = os.path.splitext(doc['Key'])[1][1:]
    fileobj = s3.get_object(Bucket=bucket_name, Key=doc['Key'])

In [93]:
filedata = fileobj['Body'].read()

In [95]:
doc['Key']

'public/07c5b42c-5847-4a61-8825-426d28e06ed0/UT Contract - Molina - January 2017.pdf'

In [96]:
with open('local_file', 'wb') as f:
    s3.download_fileobj(bucket_name, doc['Key'],f)

In [109]:
time.ctime(os.path.getmtime('local_file'))

'Tue Feb  1 20:12:29 2022'

In [110]:
statinfo = os.stat('local_file')

In [114]:
time.ctime(statinfo.st_ctime)

'Tue Feb  1 20:12:29 2022'

### Examine Current ES Index
This index was created when the DUS application was initially deployed. Under "properties" are the features in which the documents are indexed by. In this example, there are a mix of numbers, authors, and sentence fragments. The numbers correspond to ICD10 numbers since those were used to when building the index. The other properties are extracted by ES to best optimize searches across the documents in the index.

In [69]:
import json

ind_client = IndicesClient(es)
if ind_client.exists(index="textract"):
    print("The index textract exists...\n")
    index = ind_client.get(index="textract")
    print(json.dumps(index,sort_keys=True, indent=2))
else:
    print("The index textract does not exist...")

The index textract exists...

{
  "textract": {
    "aliases": {},
    "mappings": {
      "properties": {
        "5": {
          "properties": {
            " this authorization shall be in force and effect until 12/31/2020 (date event), at which time this authorization expires": {
              "fields": {
                "keyword": {
                  "ignore_above": 256,
                  "type": "keyword"
                }
              },
              "type": "text"
            }
          }
        },
        "anatomy": {
          "fields": {
            "keyword": {
              "ignore_above": 256,
              "type": "keyword"
            }
          },
          "type": "text"
        },
        "bucket": {
          "fields": {
            "keyword": {
              "ignore_above": 256,
              "type": "keyword"
            }
          },
          "type": "text"
        },
        "by": {
          "fields": {
            "keyword": {
              "ignore_abo

In [70]:
index['textract']['mappings']['properties'].keys()

dict_keys(['5', 'anatomy', 'bucket', 'by', 'christa capozzola', 'circular', 'commercial_item', 'content', 'current pharmacy name', 'current physician name', 'date', 'date of last update', 'description', 'documentId', 'event', 'from', 'heart attack [https', 'link to treatment plan problem', 'location', 'medical_condition', 'medication', 'muscle disorders [https', 'muscular dystrophy [https', 'name', 'on', 'organization', 'other', 'patient name', 'person', 'phone', 'printed name of patient or personal representative and his or her relationship to patient', 'protected_health_information', 'quantity', 'signature of patient or personal representative', 'subject', 'target date', 'test_treatment_procedure', 'time_expression', 'title', 'to', 'treatment notes'])

In [72]:
index['textract']['mappings']['properties']['date of last update']
# so it exists... why is it not returned?

{'type': 'text',
 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}

In [76]:
index['textract']['settings']

{'index': {'number_of_shards': '2',
  'provided_name': 'textract',
  'max_result_window': '10000',
  'creation_date': '1631581097237',
  'analysis': {'filter': {'custom_stopword_filter': {'type': 'stop',
     'updateable': 'true',
     'stopwords_path': 'analyzers/F220113380'},
    'english_stopwords': {'type': 'stop', 'stopwords': '_english_'},
    'remove_extra_spaces': {'pattern': '\\s+',
     'type': 'pattern_replace',
     'replacement': ' '},
    'custom_shingle_filter': {'max_shingle_size': '3',
     'min_shingle_size': '2',
     'output_unigrams_if_no_shingles': 'true',
     'output_unigrams': 'true',
     'type': 'shingle',
     'filler_token': ''}},
   'analyzer': {'custom_analyzer': {'filter': ['asciifolding',
      'lowercase',
      'english_stopwords',
      'custom_stopword_filter',
      'custom_shingle_filter',
      'trim',
      'remove_extra_spaces',
      'unique'],
     'type': 'custom',
     'tokenizer': 'classic'}}},
  'number_of_replicas': '1',
  'uuid': 'ZvrS7

### Store Current Textract as New Index
The following cell will store the current textract index as `originaltextractindex` for safe keeping (if it doesn't already exist). We will use this copy to reindex the updated textract index.

In [ ]:
from elasticsearch.helpers import reindex

if not ind_client.exists(index="originaltextractindex"):
    
    print("Index 'originaltextractindex' does not exist, creating...")
    ind_client.create(
        index="originaltextractindex",
        body={
            "settings": {
                "index": {
                    "number_of_shards": 2
                }
            },
            "mappings":{
                "properties":{
                "date":{
                    "type": "date",
                    "format": "M'/'dd'/'yyyy||date||year||year_month||dd MMM yyyy||dd'/'MM'/'yyyy||yyyy'/'MM'/'dd||dd'/'MM'/'YY||year_month_day||MM'/'dd'/'yy||dd MMM||MM'/'yyyy||M-dd-yyyy||MM'/'dd'/'yyyy||M||d'/'MM'/'yyyy||MM'/'dd'/'yy"
                    }
                }
            }
        }
    )
        
    reindex(
        client = es, # original ES client specified in the beginning
        source_index = "textract", # original index with documents we want to copy
        target_index = "originaltextractindex" # new index where documents will be copied to
    ) 

# Build and Demonstrate Filters for Custom Analyzer
The following cells will demonstrate each filter that will be added to a custom text analyzer. The analyzer is applied to the index and will be manually specified for search queries.

For our index, we will use the classic tokenizer and the asciifolding, lowercase, stopwords, and shingle filters. Below, the impact of each filter is demonstrated on the same query string. The result of each filter will be a list of tokens.

In [28]:
query_string = "Quick Brown Foxes is. today the tESt!! but i Will not be blAh!?"

### Lowercase Filter

In [ ]:
body = {
    "tokenizer":"classic", # removes punctuation and splits string on whitespace
    "filter":[
        "asciifolding", # converts characters to ASCII
        "lowercase", # transforms characters to lowercase
    ],
    "text" : query_string
}

# store index client class as a variable
ind_client = IndicesClient(es)

# use the analyze module of the index class to ping the ES/analyze API
token_dict = ind_client.analyze(
    body=body, # request body
    index="originaltextractindex" # index name
)

# extract tokens from result dictionary
tokens = []
for token in token_dict["tokens"]:
    tokens.append(token["token"])
tokens

### Custom and Existing Stopword Filter
In this example, a custom and existing stopword filter are applied to the text. With a custom stopword filter, words that are specific to the use case can be added to a custom dictionary for removal from index and query text.

Before moving onto the next step, import a stopwords dictionary into AWS ES. First, add a TXT-file that contains your list of stopwords to an S3 bucket. Each stopword has to be on a separate line within the TXT-file. Next, navigate to the ES console and select "Packages" within the left-hand panel. Select "Import" and point to the location of your stopwords file in the S3 bucket. Below is a screenshot of the import package dashboard.

<img src="assets/stopwords_package.png" /><br><br>


Once your package is loaded into ES, AWS ES will generate an ID for your package. Take note of this ID since it will be used to reference your list of stopwords in future API calls.

<img src="assets/stopwords_package_dashboard.png" />

In [ ]:
body = {
    "tokenizer":"classic",# removes punctuation and splits string on whitespace
    "filter":[
        {# ES offered filter: English stopwords
            "type":"stop", # specify the type of the filter as stop for stopword filter
            "stopwords":"_english_" # stopword filter will use English stopwords
        },
        {# custom filter: stop words based upon custom dictionary uploaded onto AWS ES
            "type":"stop",
            "stopwords_path":"analyzers/F220113380", # ID of the custom dictionary in AWS ES
            "updateable":True
        },
    ],
    "text" : query_string
}

# use the analyze module of the index class to ping the ES/analyze API
token_dict = ind_client.analyze(
    body=body, # request body
    index="originaltextractindex" # index name
)

# extract tokens from result dictionary
tokens = []
for token in token_dict["tokens"]:
    tokens.append(token["token"])
tokens

### Shingles Filter
Shingles are synonomous with the popular definition of ngrams. In ElasticSearch, ngrams are used for predictive text purposes and create ngrams at the character level instead of the word level. Shingles in ElasticSearch create ngrams at the word level which is what we want.

In [15]:
body = {
    "tokenizer":"classic",# removes punctuation and splits string on whitespace
    "filter":[
        {# existing filter: shingles which are synonomous with the popular definition of ngram
            "type":"shingle", # specify filter type as a shingle
            "min_shingle_size":2, # min size=2 for bigram
            "max_shingle_size":3, # mac size=3 for trigram
            "output_unigrams": True,
            "output_unigrams_if_no_shingles":True
        },
    ],
    "text" : query_string
}

# use the analyze module of the index class to ping the ES/analyze API
token_dict = ind_client.analyze(
    body=body, # request body
    index="originaltextractindex" # index name
)

# extract tokens from result dictionary
tokens = []
for token in token_dict["tokens"]:
    tokens.append(token["token"])
tokens

['Quick',
 'Quick Brown',
 'Quick Brown Foxes',
 'Brown',
 'Brown Foxes',
 'Brown Foxes is',
 'Foxes',
 'Foxes is',
 'Foxes is today',
 'is',
 'is today',
 'is today the',
 'today',
 'today the',
 'today the tESt',
 'the',
 'the tESt',
 'the tESt but',
 'tESt',
 'tESt but',
 'tESt but i',
 'but',
 'but i',
 'but i Will',
 'i',
 'i Will',
 'i Will not',
 'Will',
 'Will not',
 'Will not be',
 'not',
 'not be',
 'not be blAh',
 'be',
 'be blAh',
 'blAh']

### Combining all Filters
Below, all filters are combined to create a single custom filter. Each individual filter will be executed in the order in which it appears below.

Note: The shingles filter appears after the stopword filter. In ES, stopwords are replaced by an underscore by default and are not explicitly removed. Hence, there are underscores within the bi- and tri-grams below. In the following section, we will replace the underscore with an empty string.

In [16]:
body = {
    "tokenizer":"classic",# removes punctuation and splits string on whitespace
    "filter":[
        "asciifolding", # converts characters to ASCII
        "lowercase", # transforms characters to lowercase
        {# ES offered filter: English stopwords
            "type":"stop", # specify the type of the filter as stop for stopword filter
            "stopwords":"_english_" # stopword filter will use English stopwords
        },
        {# custom filter: stop words based upon custom dictionary uploaded onto AWS ES
            "type":"stop",
            "stopwords_path":"analyzers/F220113380", # ID of the custom dictionary in AWS ES
            "updateable":True
        },
        {# existing filter: shingles which are synonomous with the popular definition of ngram
            "type":"shingle", # specify filter type as a shingle
            "min_shingle_size":2, # min size=2 for bigram
            "max_shingle_size":3, # mac size=3 for trigram
            "output_unigrams": True,
            "output_unigrams_if_no_shingles":True
        },
        "trim", #remove extra spaces from the beginning and end created by shingle filter
        {#ES offered filter: remove extra whitespace in between words caused by shingle filter
            "type":"pattern_replace", # specify filter type as REGEX replacement
            "pattern":"\\s+", # search for consecutive spaces
            "replacement":" " # replace consecutive spaces with a single space
        },
        "unique" # remove duplicates that are introduced by shingles processing and cleaning
    ],
    "text" : query_string
}

# use the analyze module of the index class to ping the ES/analyze API
token_dict = ind_client.analyze(
    body=body, # request body
    index="originaltextractindex" # index name
)

# extract tokens from result dictionary
tokens = []
for token in token_dict["tokens"]:
    tokens.append(token["token"])
tokens

['quick',
 'quick brown',
 'quick brown foxes',
 'brown',
 'brown foxes',
 'brown foxes _',
 'foxes',
 'foxes _',
 'foxes _ today',
 '_ today',
 '_ today _',
 'today',
 'today _',
 'today _ test',
 '_ test',
 '_ test _',
 'test',
 'test _',
 'test _ _',
 '_ _ blah',
 '_ blah',
 'blah']

# Create New Index

### Delete "textract" Index
Since the `textract` namespace is used throughout multiple Lambda functions, it is easier to create a new index called `textract` than it is to change the index name for all of the Lambda functions that require it. The following cells will delete the existing index and then reindex it using `textractedit`.

The following cell confirms that `textract` exists.

In [17]:
ind_client.get_alias("*")

{'originaltextractindex': {'aliases': {}},
 '.kibana_1': {'aliases': {'.kibana': {}}},
 'textract': {'aliases': {}}}

In [18]:
ind_client.delete(index="textract")

{'acknowledged': True}

The next cell will confirm that `textract` is no longer an index in our ES domain.

In [19]:
ind_client.get_alias("*")

{'originaltextractindex': {'aliases': {}},
 '.kibana_1': {'aliases': {'.kibana': {}}}}

### Create Index
Using the index client that we specified earlier, we will create a new index with the `textract` name space.

In [21]:
ind_client.create(
    index="textract",
    body={
        "settings": {
            "index": {
                "number_of_shards": 2, # number of times the index volume is divided into smaller parts
                "max_result_window" : 10000, # max number of results to return, if applicable
                "analysis" : {
                    "analyzer":{
                        # specify the names of components of analyzer here. will define below if custom
                        "custom_analyzer":{
                            "type":"custom",
                            "tokenizer":"classic",# splits on whitespace, removes punctuation
                            "filter":[
                                 # converts characters to ASCII
                                "asciifolding",
                                
                                # makes character lowercase
                                "lowercase",
                                
                                # ES filter: English stopwords; requires some customization (below)
                                "english_stopwords",
                                
                                # custom filter: stop words based upon custom dictionary uploaded onto AWS ES
                                "custom_stopword_filter",
                                
                                # ES filter: shingles which are synonomous with the popular definition of ngram; requires some customization (below)
                                "custom_shingle_filter",
                                
                                # ES filter: remove extra spaces from the beginning and end created by shingle filter
                                "trim",
                                
                                # ES filter: remove extra whitespace in between words caused by shingle filter; requires some customization (below)
                                "remove_extra_spaces",
                                
                                # ES filter: remove duplicates that are introduced by shingles processing and cleaning
                                "unique"
                            ]
                        }
                    },
                    # define custom and ES filters
                    "filter":{
                        "custom_stopword_filter":{
                            "type":"stop",
                            "stopwords_path":"analyzers/F220113380",
                            "updateable":True
                        },
                        "english_stopwords":{
                            "type":"stop",
                            "stopwords":"_english_"
                        },
                        "custom_shingle_filter":{
                            "type":"shingle",
                            "min_shingle_size":2,
                            "max_shingle_size":3,
                            "output_unigrams": True,
                            "output_unigrams_if_no_shingles":True,
                            "filler_token":"" # replace the default underscore placeholder with an empty string
                        },
                        "remove_extra_spaces":{
                            "type":"pattern_replace",
                            "pattern":"\\s+",
                            "replacement":" "
                        }
                    }
                }
            }
        },
        # create data fields that are added to the index after being extracted by ES
        "mappings":{
            "properties":{
                # add a data field to index
                "date":{
                    # specify mapping type as date
                    "type": "date",
                    
                    # specify the format of the mapping type. ES looks for dates that follow these formats
                    "format": "M'/'dd'/'yyyy||date||year||year_month||dd MMM yyyy||dd'/'MM'/'yyyy||yyyy'/'MM'/'dd||dd'/'MM'/'YY||year_month_day||MM'/'dd'/'yy||dd MMM||MM'/'yyyy||M-dd-yyyy||MM'/'dd'/'yyyy||M||d'/'MM'/'yyyy||MM'/'dd'/'yy"
                }
            }
        }
    }
)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'textract'}

The following cell confirms that `textract` is listed as an index again.

In [ ]:
ind_client.get_alias("*")

# Add Documents to New Index
Documents can be added to a new index by copying documents from another index. This is achieved via `reindex`.

In [23]:
from elasticsearch.helpers import reindex

reindex(
    client = es, # original ES client specified in the beginning
    source_index = "originaltextractindex", # original index with documents we want to copy
    target_index = "textract" # new index where documents will be copied to
)

(7, 0)

# Verify Changes to Index
When creating the index above, we specified our collection of text processors with the name `custom_analyzer`. In the cells below, we can reference this analyzer by name when using the `textract` index.

In [24]:
body = {
    # evoking our custom analyzer from the index
    "analyzer" : "custom_analyzer",
    "text" : query_string
}


token_dict = ind_client.analyze(
    body=body,
    index="textract"
)

tokens = []
for token in token_dict["tokens"]:
    tokens.append(token["token"])
tokens

['quick',
 'quick brown',
 'quick brown foxes',
 'brown',
 'brown foxes',
 'foxes',
 'foxes today',
 'today',
 'today test',
 'test',
 'blah']

Now, let's test the new index on an old query. The output are highlighted fragments from each source in the form of a list. You can see that the results have improved significantly and there are fewer references to kidney related documents.

In [25]:
keyword = "What is diabetes\?"

ES_HIGHLIGHT_FRAGMENT_SIZE = 200
host = "vpc-dusstac-dussta-h8vo3o6f68eb-knopiij3g5xsdg42i2sxvarih4.us-east-1.es.amazonaws.com" #ES domain

searchBody = {
     "query" : {
        "query_string": {
            "query": keyword,
            "fields":["content^3","name","keyword^2","title^4","summary"],
            "fuzziness":"AUTO",
            "analyzer":"custom_analyzer",
           }
      },
      "highlight" : {
        "fields" : {
            "content" : { "pre_tags" : [""], "post_tags" : [""] },
          },
          "fragment_size" : ES_HIGHLIGHT_FRAGMENT_SIZE,
          "require_field_match": False
     }
}


output = es.search(
    index='textract',
    size=1000,
    body=searchBody,
    _source = True,
    filter_path=['hits.hits._id', 'hits.hits._source','hits.hits.highlight','hits.hits._score'],
    request_timeout=5
)



n_results = len(output["hits"]["hits"])
print(f"The Elasticsearch query returned {n_results} results.")

print(f"Example output:\n\n")

for x in output["hits"]["hits"]:
    print(x["highlight"]["content"],"\n")

The Elasticsearch query returned 1 results.
Example output:


['Preventing Diabetes Problems I NIDDK\n10/23/20,3127 PM\nPreventing Diabetes Problems\nView or Print All Sections\n>\nHeart Disease & Stroke\nLow Blood Glucose\n(Hypoglycemia)\nDiabetes can damage blood vessels', 'Managing your diabetes can\nyour kidneys by managing your diabetes\nhelp prevent nerve damage that affects\nand meeting your blood pressure goals.\nyour feet and limbs, and organs such as\nyour heart.', 'Disease\nDiabetes can cause nerve damage and\nDiabetes can damage your eyes and lead\npoor blood flow, which can lead to\nto low vision and blindness.', '&\nDementia &\nSleep Apnea &\nDiabetes\nDiabetes\nDiabetes\nDiabetes\nDepression NIH\nDiabetes is linked\nHigh blood\nPeople who have\nis common among\nto some types of\nglucose increases\nsleep apnea NIH\npeople with', 'https://www.niddk.nih.gov/health-information/diabetes/overview/preventing-problems\nPage 3 of 4\n\nPreventing Diabetes Problems I NIDDK\n10/23/2

With another search, you can see how specific the results are to the phrases in the query string.

In [26]:
keyword = "dibetes in older patients"

ES_HIGHLIGHT_FRAGMENT_SIZE = 200
host = "vpc-dusstac-dussta-h8vo3o6f68eb-knopiij3g5xsdg42i2sxvarih4.us-east-1.es.amazonaws.com" #ES domain

searchBody = {
     "query" : {
        "query_string": {
            "query": keyword,
            "fields":["content^3","name","keyword^2","title^4","summary"],
            "fuzziness":"AUTO",
            "analyzer":"custom_analyzer",
           }
      },
      "highlight" : {
        "fields" : {
            "content" : { "pre_tags" : [""], "post_tags" : [""] },
          },
          "fragment_size" : ES_HIGHLIGHT_FRAGMENT_SIZE,
          "require_field_match": False
     }
}


output = es.search(
    index='textract',
    size=1000,
    body=searchBody,
    _source = True,
    filter_path=['hits.hits._id', 'hits.hits._source','hits.hits.highlight','hits.hits._score'],
    request_timeout=5
)



n_results = len(output["hits"]["hits"])
print(f"The Elasticsearch query returned {n_results} results.")

print(f"Example output:\n\n")

for x in output["hits"]["hits"]:
    print(x["highlight"]["content"],"\n")

The Elasticsearch query returned 2 results.
Example output:


['The NIDDK translates and disseminates research findings to\nincrease knowledge and understanding about health and disease among patients, health professionals, and the\npublic.'] 

['Available\nfrom:\nittps://www.cedars-sinai.edu/Patients/Health-Conditions/Neuromuscular-Disorders.aspx [https://www.cedars-sinai.edu/Patients/\nealth-Conditions/Neuromuscular-Disorders.aspx]\n2.'] 

